In [327]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as grb
from gurobipy import GRB

# Input Problem Parameters

## Définissons les variables

In [328]:
from extract_parameters import extract_parameters

################################
######## TO DO #################
################################

# choose taille among: "small", "medium", "large"
taille = "small"

In [329]:
################################
######## DO NOT CHANGE #########
################################
conge, qualif, d, p, b, besoin = extract_parameters(taille)

In [330]:
print("conge : ",conge)
print("qualif : ",qualif)
print("date livraison : ",d)
print("pénalité : ",p)
print("bénéfice : ",b)
print("besoin : ",besoin)

conge :  [[1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 1, 1, 1]]
qualif :  [[0, 1, 1], [0, 1, 1], [1, 0, 1]]
date livraison :  [3, 3, 4, 3, 5]
pénalité :  [3, 3, 3, 3, 3]
bénéfice :  [20, 15, 15, 20, 10]
besoin :  [[1, 1, 1, 0, 0], [1, 2, 0, 2, 0], [1, 0, 2, 1, 2]]


-- Correction -- début --

NB : ordre des employés inverse par rapport au .json (par ex. indice 2 correspond à Olivia dans l'instance small)
NB2 : ordre des qualifications et des projets comme dans le .json
NB3 : conge décrit par employé puis par projet si employé dispo ou pas
NB4 : qualif décrit par qualif puis par employé si les qualif sont compatibles
NB5 : besoin décrit par qualif puis par projet combien de jours sont nécessaires

-- Correction -- fin --

## Définissons les paramètres

In [331]:
from define_nums import define_nums

nombre_employes, horizon, nombre_qualif, nombre_projets = define_nums(conge, qualif, d, p, b, besoin)

In [332]:
print("nombre_employes : ",nombre_employes)
print("horizon : ",horizon)
print("nombre_qualif : ",nombre_qualif)
print("nombre_projets : ",nombre_projets)

nombre_employes :  3
horizon :  5
nombre_qualif :  3
nombre_projets :  5


In [333]:
## u_j, le projet j est réalisé
u = [0]*nombre_projets
u_shape = len(u)

## x_i_j_k_n, l'employé k travaille sur le projet j avec la qualif i
## au jour n
#x = [[[[0]*horizon for _ in range(nombre_employes)] for _ in range(nombre_projets)] for _ in range(nombre_qualif)]
x_shape = [(i,j,k, n) for i in range(nombre_qualif) for j in range(nombre_projets) for k in range(nombre_employes) for n in range(horizon)]

## t_k_j, l'employé k travaille sur le projet j
#t = [[0]*nombre_projets for _ in range(nombre_employes)]
t_shape = [(i,j) for i in range(nombre_employes) for j in range(nombre_projets)]

## begin_j, la date de début effective
begin = [0]*nombre_projets
begin_shape = len(begin)

## end_j, la date de fin effective = date de livraison du projet j
end = [0]*nombre_projets
end_shape = len(end)

## delay_j, le nombre de jours de retard par projet j
delay = [0]*nombre_projets
delay_shape = len(delay)

## longueur_j, la longueur du projet j
longueur = [0]*nombre_projets
longueur_shape = len(longueur)

## projet_par_employe_k, le nombre de projet par employe
projet_par_employe = [0]*nombre_employes
projet_par_employe_shape = len(projet_par_employe)

# majorant pour la contrainte de couverture
maj = horizon * nombre_employes


job_in_process_shape = [(j, n) for j in range(nombre_projets) for n in range(horizon)]

# Model deployment

## Add the variables

In [334]:
model = grb.Model("Planning")

# L'affectation du planning, la variable clé
x = model.addVars(x_shape, vtype=GRB.BINARY, name="x")

## Les dates de fin des projets
dateFin = model.addVars(end_shape, lb = 0, ub = horizon, name = "dateFin")

## Les dates de début des projets
dateDebut = model.addVars(begin_shape, lb = 0, ub = horizon, name = "dateDebut")

## Le nombre de jours de delai
delay = model.addVars(delay_shape, lb = 0, ub = horizon, name = "delay")

## Les longueurs des projets
# -- Correction -- début --
# Rq : il y a déjà une variable (pas une variable de décision, une variable au sens programmation) qui s'appelle longueur, bien veiller à ce que cela ne soit pas un pb
longueur = model.addVars(longueur_shape, lb = 0, ub = horizon, name = "longueur")
# -- Correction -- fin --

## Nombre de projet par employé
projet_par_employe = model.addVars(projet_par_employe_shape, lb = 0, ub = nombre_projets, name = "projet_par_employe")

## L'affectation sur un projet d'un employé
affectation = model.addVars(t_shape, vtype = GRB.BINARY, name = "affectation")

## La réalisation ou non d'un projet
u = model.addVars(u_shape, vtype = GRB.BINARY, name="u")

## Verification du projet
job_in_process = model.addVars(job_in_process_shape, vtype = GRB.BINARY, name = "job_in_process")

In [335]:
# Update le modele pour confirmer la création des variables
model.update()


## Add constraints

In [336]:
for i in range(nombre_qualif):
    for j in range(nombre_projets):
        for k in range(nombre_employes):
            for n in range(horizon):
                ## La qualification du personne
                model.addConstr( x[i,j,k,n] <= qualif[i][k], name="QualificationPersonnel[{0}{1}{2}{3}]".format(i,j,k,n))
                model.addConstr( x[i,j,k,n] <= (besoin[i][j] >=1), name = "BesoinQualifPersonnel[{0}{1}{2}{3}]".format(i,j,k,n))
                ## Contraintes des congés
                model.addConstr( x[i,j,k,n] <= conge[k][n], name=f"CongesPersonnel[{i}{j}{k}{n}]")
        
        ## Contrainte d'unicité de réalisation d'un projet
        # -- Correction -- début --
        # Rq : pour éviter que des employés ne soient staffés sur des projets qui finalement ne seront pas réalisés pcq toutes les qualifications ne sont pas couvertes,
        # il vaut mieux mettre une contrainte d'égalité plutôt que des inégalités
        # model.addConstr(grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) >= besoin[i][j]*u[j], name=f"UniciteRealisationMin[{i}{j}]")
        # model.addConstr(grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) <= besoin[i][j], name=f"UniciteRealisation[{i}{j}]")
        model.addConstr(grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) == besoin[i][j]*u[j], name=f"UniciteRealisation[{i}{j}]")
        # -- Correction -- fin --

        ## Couverture des qualifications
        # -- Correction -- début --
        # Rq : les contraintes ci-dessous (avec le reste des contraintes) engendrent l'infaisabilité du modèle
        # Pour la première contrainte, si la tâche j n'est pas faite, comme maj vaut (horizon x nb employés), la contrainte oblige à ce que les x[i,j,k,n] (à i, j fixés) valent tous 1
        # ce qui, d'une part, n'a pas vraiment de sens et, d'autre part, est incompatible avec d'autres contraintes sur les x[i,j,k,n] telles que les contraintes de congés
        # Pour la deuxième contrainte, si la tâche j n'est pas faite, la contrainte oblige à ce que la somme des x[i,j,k,n] (à i, j fixés) soient négatives,
        # ce qui, d'une part, n'a pas vraiment de sens et, d'autre part, est incompatible avec le fait que les les x[i,j,k,n] soient des variables binaires (donc positives)
        # model.addConstr( grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) >= maj*(1-u[j]), name=f"CouvertureQualificationsInf[{i}{j}]") 
        # model.addConstr( grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) <= maj*u[j] -10**-4, name=f"CouvertureQualificationsSup[{i}{j}]")
        # -- Correction -- fin --


## Unicité de l'affectation quotidienne
for k in range(nombre_employes):
    for n in range(horizon):
        model.addConstr(grb.quicksum(grb.quicksum(x[i,j,k,n] for j in range(nombre_projets)) for i in range(nombre_qualif)) <= 1, name=f"UniciteAffectation[{k}{n}]")

        
for j in range(nombre_projets):
    for n in range(horizon):
        model.addConstrs(((job_in_process[j,n] >= x[i,j,k,n]) for i in range(nombre_qualif) for k in range(nombre_employes)), name=f"ContrainteJobInProcess[{j}{n}]")
    # start date
    # -- Correction -- début --
    # Rq : avec la contraite ci-dessous, dateDebut[j] n'a pas le sens désiré (cf. valeurs affichées après résolution lorsque vous ferez du multi-objectif),
    # une petite modification doit être apportée
    model.addConstrs(((dateDebut[j] <= job_in_process[j,n]) for n in range(horizon)), name = f"ContrainteDateDebut[{j}]")
    # -- Correction -- fin --
    ## end date
    # -- Correction -- début --
    # Rq : avec la contraite ci-dessous, dateFin[j] n'a pas le sens désiré (cf. valeurs affichées après résolution lorsque vous ferez du multi-objectif)
    # une petite modification doit être apportée
    model.addConstrs(((job_in_process[j,n] <= dateFin[j]) for n in range(horizon)), name=f"FinProjet[{j}]")
    # -- Correction -- fin --
    ## longueur
    model.addConstr(longueur[j] == dateFin[j] - dateDebut[j], name=f"LongueurProjet[{j}]")
    ## end date
    #model.addConstrs(((x[i,j,k,n] * n <= dateFin[j]) for i in range(nombre_qualif) for k in range(nombre_employes) for n in range(horizon)), name=f"FinProjet[{j}]")
    ## delay
    # -- Correction -- début --
    # Rq : contrainte pas nécessaire comme delay sont des variables entières positives
    # model.addConstr(delay[j] >= 0, name=f"DelaiProjet_positif[{j}]")
    # -- Correction -- fin --
    model.addConstr(delay[j] >= dateFin[j] - (d[j] - 1), name=f"DelaiProjet_difference[{j}]")
    #model.addConstr(delay[j]== grb.max_(0,dateFin[j] - d[j]), name=f"DelaiProjet[{j}]")
    

## projet_par_employe
for k in range(nombre_employes):
    model.addConstr(projet_par_employe[k] == grb.quicksum( affectation[k,j] for j in range(nombre_projets)), name=f"FinProjet[{j}]")
    
    
## Contrainte de l'affectation à un projet selon les jours de travail
for k in range(nombre_employes):
    for j in range(nombre_projets):
        # -- Correction -- début --
        # Rq : contrainte pas nécessaire comme affectation sont des variables binaires
        # model.addConstr(affectation[k,j] <=1, name=f"AffectationProjet_1[{k}{j}]")
        # -- Correction -- fin --
        model.addConstr(affectation[k,j] <= grb.quicksum(grb.quicksum(x[i,j,k,n] for i in range(nombre_qualif)) for n in range(horizon)), name=f"AffectationProjetQuicksum[{k}{j}]")
        #model.addConstr(affectation[k,j] == grb.min_(1,grb.quicksum(grb.quicksum(x[i,j,k,n] for i in range(nombre_qualif)) for n in range(horizon))), name=f"AffectationProjet[{j}]")

        
    

## Fonctions objectif

In [337]:
## bénéfice total
benefice_tot = grb.quicksum( (b[j]*u[j] - delay[j]*p[j]) for j in range(nombre_projets) )

# -- Correction -- début --
# Rq : je commente ces lignes pour me focaliser juste sur le premier objectif à savoir le bénéfice total
## Nombre de projet maximum par personne
# projet_max = model.addVar(name="projet_max")
# model.addConstrs(projet_max>=projet_par_employe[k] for k in range(nombre_employes))
# model.update()

## Compacité
# longueur_max = grb.max_(longueur)

# model.ModelSense = GRB.MINIMIZE
# model.setObjectiveN(benefice_tot,0,2)
#model.setObjectiveN(projet_max, 1, 1)
#model.setObjectiveN(longueur_max, 2, 0)
# -- Correction -- fin --

# -- Correction -- début --
# Rq : j'ajoute ces lignes pour me focaliser juste sur le premier objectif à savoir le bénéfice total
model.ModelSense = GRB.MAXIMIZE
model.setObjective(benefice_tot)
model.ModelSense = GRB.MAXIMIZE
# -- Correction -- fin --

In [338]:
### Corriger les fonctions obj 2 et 3 qui plantent
### Comprendre pq on passe jamais nos x à 1

# Model optimisation

In [339]:
# Verify model formulation

model.write('marketSharing.lp')

# Run optimization engine

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1013 rows, 294 columns and 1973 nonzeros
Model fingerprint: 0x35c78fb6
Variable types: 23 continuous, 271 integer (270 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 5e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 855 rows and 140 columns
Presolve time: 0.00s
Presolved: 158 rows and 154 columns
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 () 

In [340]:
# # Print the values of all variables
# for v in model.getVars():
#     print(f"{v.VarName} = {v.X}")

# -- Correction -- début --
# Rq : pour voir quels sont les projets faits
# for j in range(nombre_projets):
#     v = model.getVarByName(f"u[{j}]")
#     if int(v.x) == 1:
#         print(f"la tâche {j} est faite")
#     else:
#         print(f"la tâche {j} n'est pas faite")
# -- Correction -- fin --

# -- Correction -- début --
# Rq : pour voir quels sont les edt des employés et le détail de la réalisation de chaque tâche (de façon ordonnée)
schedules = []
tasks_performances = []
for i in range(nombre_qualif):
    for j in range(nombre_projets):
        for k in range(nombre_employes):
            for n in range(horizon):
                v = model.getVarByName(f"x[{i},{j},{k},{n}]")
                if int(v.x) == 1:
                    schedules.append((k, n, j, i))
                    tasks_performances.append((j, i, k, n))
schedules.sort()
for (k, n, j, i) in schedules:
    print(f"l'employé {k} fait, à la date {n}, la tâche {j} pour la qualif {i}")
tasks_performances.sort()
for (j, i, k, n) in tasks_performances:
    print(f"la tâche {j} pour la qualif {i} est faite par {k} à la date {n}")
# -- Correction -- fin --

# -- Correction -- début --
# Rq : pour voir quelles sont les dates de début et de fin des tâches réalisees 
# (à n'étudier que lorsque longueur_max est mis en objectif, sinon les variables dateDebut et dateFin ne sont pas assez contraintes pour prendre exactement le sens souhaité)
for j in range(nombre_projets):
    v = model.getVarByName(f"u[{j}]")
    v1 = model.getVarByName(f"dateDebut[{j}]")
    v2 = model.getVarByName(f"dateFin[{j}]")
    if int(v.x) == 1:
        print(f"la tâche {j} commence à la date {int(v1.x)} et finit à la date {int(v2.x)}")
# -- Correction -- fin --

l'employé 0 fait, à la date 0, la tâche 0 pour la qualif 2
l'employé 0 fait, à la date 1, la tâche 3 pour la qualif 2
l'employé 0 fait, à la date 3, la tâche 2 pour la qualif 2
l'employé 0 fait, à la date 4, la tâche 4 pour la qualif 2
l'employé 1 fait, à la date 0, la tâche 0 pour la qualif 0
l'employé 1 fait, à la date 2, la tâche 3 pour la qualif 1
l'employé 1 fait, à la date 3, la tâche 2 pour la qualif 0
l'employé 2 fait, à la date 0, la tâche 0 pour la qualif 1
l'employé 2 fait, à la date 1, la tâche 3 pour la qualif 1
l'employé 2 fait, à la date 3, la tâche 2 pour la qualif 2
l'employé 2 fait, à la date 4, la tâche 4 pour la qualif 2
la tâche 0 pour la qualif 0 est faite par 1 à la date 0
la tâche 0 pour la qualif 1 est faite par 2 à la date 0
la tâche 0 pour la qualif 2 est faite par 0 à la date 0
la tâche 2 pour la qualif 0 est faite par 1 à la date 3
la tâche 2 pour la qualif 2 est faite par 0 à la date 3
la tâche 2 pour la qualif 2 est faite par 2 à la date 3
la tâche 3 pour

In [ ]:
model.computeIIS()

In [147]:
model.objVal

817.0

In [248]:
x_values = [var.x for var in model.getVars()]
print(x_values)
print(horizon*nombre_projets*nombre_employes*nombre_qualif)
print(nombre_projets*(2+nombre_employes))

AttributeError: Unable to retrieve attribute 'x'